
### Supervised Learning
### Activity: Building a Student Intervention System

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail or pass. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** 

# classification
it has categotical answer

### Question-2
load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Read student data
df=pd.read_csv('student-data.csv')
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


### Question-3
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# Calculate number of students
n_students = df.shape[0]

In [4]:
# Calculate number of features
n_features = df.shape[1]

In [5]:
# Calculate passing students
n_passed = df['passed'].loc[df['passed'] == 'yes'].count()

In [6]:
# Calculate failing students
n_failed = df['passed'].loc[df['passed'] == 'no'].count()

In [7]:
# Calculate graduation rate
grad_rate = (n_passed/n_students) * 100

In [8]:
# Print the results
print("\nTotal number of students in the dataset: ", n_students)
print("\nTotal number of features for each student in the dataset: ", n_features)
print("\nTotal number of students who passed: ", n_passed)
print("\nTotal number of students who failed: ", n_failed)
print("\nGraduation rate of the class is: ", grad_rate, 3)


Total number of students in the dataset:  395

Total number of features for each student in the dataset:  31

Total number of students who passed:  265

Total number of students who failed:  130

Graduation rate of the class is:  67.08860759493672 3


## Preparing the Data
you will prepare the data for modeling, training and testing.

### Question-4 Identify feature and target columns


separate the student data into feature and target columns to see if any features are non-numeric.

In [9]:
# Extract feature columns

In [10]:
df.drop('passed',axis=1)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,yes,no,no,5,5,4,4,5,4,11
391,MS,M,17,U,LE3,T,3,1,services,services,...,yes,yes,no,2,4,5,3,4,2,3
392,MS,M,21,R,GT3,T,1,1,other,other,...,yes,no,no,5,5,3,3,3,3,3
393,MS,M,18,R,LE3,T,3,2,services,other,...,yes,yes,no,4,4,1,3,4,5,0


In [11]:
# Extract target column 'passed'

In [12]:
df['passed']

0       no
1       no
2      yes
3      yes
4      yes
      ... 
390     no
391    yes
392     no
393    yes
394     no
Name: passed, Length: 395, dtype: object

In [13]:
# Separate the data into feature data and target data (X and y, respectively)

In [14]:
X=df.drop(['passed'],axis=1)
y=df['passed']

### Question-5 Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [15]:
#one hot encoding
X = pd.get_dummies(X)
X.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,1,0,0,1,0,1,1,0,1,0
1,17,1,1,1,2,0,5,3,3,1,...,1,0,1,0,0,1,0,1,1,0
2,15,1,1,1,2,3,4,3,2,2,...,1,0,0,1,0,1,0,1,1,0
3,15,4,2,1,3,0,3,2,2,1,...,0,1,0,1,0,1,0,1,0,1
4,16,3,3,1,2,0,4,3,2,1,...,1,0,0,1,0,1,1,0,1,0


### Question - 6 Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. you will need to implement the following:
- Randomly shuffle and split the data (`X`, `y`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [16]:
# splitting the data into train and test


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.24)

In [18]:
# Show the results of the split
print("\nTraining feature set size: ", X_train.shape)
print("\nTraining target set size: ", y_train.shape)
print("\nTesting feature set size: ", X_test.shape)
print("\nTesting target set size: ", y_train.shape)


Training feature set size:  (300, 56)

Training target set size:  (300,)

Testing feature set size:  (95, 56)

Testing target set size:  (300,)


### Question - 7  Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data and measure the accuracy score.

Logistic Regression
Support Vector Machine
Decision Tree 

###  Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

#explaination
Logistic Regression

Logistic regression is the classification counterpart to linear regression. 
Predictions are mapped to be between 0 and 1 through the logistic function, which means that predictions can be interpreted as class probabilities.
The models themselves are still "linear," so they work well when your classes are linearly separable.

Pros: Outputs have a nice probabilistic interpretation, and the algorithm can be regularized to avoid overfitting.
      Logistic models can be updated easily with new data using stochastic gradient descent.

Cons : Logistic regression tends to underperform when there are multiple or non-linear decision boundaries.
      They are not flexible enough to naturally capture more complex relationships.

SVM (SUPPORT VECTOR MACHINE)

Support vector machines (SVMs) are powerful yet flexible supervised machine learning algorithms which are used both for classification and regression.
But generally, they are used in classification problems. SVMs have their unique way of implementation as compared to other machine learning algorithms.
Lately, they are extremely popular because of their ability to handle multiple continuous and categorical variables.

Pros: SVM classifiers offers great accuracy and work well with high dimensional space.
      SVM classifiers basically use a subset of training points hence in result uses very less memory.
Cons: They have high training time hence in practice not suitable for large datasets.
      Another disadvantage is that SVM classifiers do not work well with overlapping classes.        

Decision Tree
Decision tree learning or induction of decision trees is one of the predictive modelling approaches used in statistics, data mining and machine learning.
It uses a decision tree (as a predictive model) to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves).

Pros:
1.Comprehensive: It takes consideration of each possible outcome of a decision and traces each node to the conclusion accordingly.
2.Specific: Decision Trees assign a specific value to each problem, decision, and outcome(s). It reduces uncertainty and ambiguity and also increases clarity.
3.Simplicity: Decision Tree is one of the easier and reliable algorithms as it has no complex formulae or data structures.
Only simple statistics and maths are required for calculation.
4.Versatile: Decision Trees can be manually constructed using maths and as well be used with other computer programs.

Cons:    
    1. Decision trees are less appropriate for estimation and financial tasks where we need an appropriate value(s).
    2.It is an error-prone classification algorithm as compared to other computational algorithms.
    3.It is sometimes unstable as small variations in the data set might lead to the formation of a new tree.

In [20]:
# Import the three supervised learning models from sklearn

from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Logistic Regression

In [21]:
# fit model-1  on traning data 

In [38]:
from sklearn.linear_model import LogisticRegression
log_r=LogisticRegression()
log_r.fit(X_train,y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
# predict on the test data 

In [39]:
y_predict=log_r.predict(X_test)

In [40]:
# calculate the accuracy score

In [41]:
print("Accuracy is:",accuracy_score(y_test,y_predict))

Accuracy is: 0.7157894736842105


# SVM

In [29]:
# fit the model-2 on traning data and predict on the test data and measure the accuracy

In [30]:
svm_linear=SVC(kernel='linear')
svm_linear.fit(X_train,y_train)
y_pred=svm_linear.predict(X_test)

In [31]:
print("Accuracy is:",accuracy_score(y_test,y_pred))

Accuracy is: 0.6947368421052632


# Decision Tree

In [32]:
# fit the model-3 on traning data and predict on the test data and measure the accuracy

In [33]:
from sklearn.tree import DecisionTreeClassifier
dt_mod = DecisionTreeClassifier()
dt_mod.fit(X_train, y_train)
y_predictions = dt_mod.predict(X_test)

In [34]:
print("Accuracy is :",accuracy_score(y_test,y_predictions))

Accuracy is : 0.6105263157894737


Insight
Accuracy is the heighest for Logistic Regression and hence it is the best model in the 3 for this case